# Curves

### CompositeCurve example

The first section here regards efficient operations and compositing two curves.

In [1]:
from rateslib import dt
from rateslib.curves import Curve, LineCurve, CompositeCurve

In [2]:
line_curve1 = LineCurve({dt(2022, 1, 1): 2.0, dt(2022, 1, 3): 4.0}, id="C1_")
line_curve2 = LineCurve({dt(2022, 1, 1): 0.5, dt(2022, 1, 3): 1.0}, id="C2_")
composite_curve = CompositeCurve(curves=(line_curve1, line_curve2))
composite_curve.rate(dt(2022, 1, 2))

3.75

In [3]:
line_curve1._set_ad_order(1)
line_curve2._set_ad_order(1)
composite_curve.rate(dt(2022, 1, 2))

<Dual: 3.750000, ('C1_0', 'C1_1', 'C2_0', 'C2_1'), [0.5 0.5 0.5 0.5]>

The code above demonstrates the summing of individual rates and of interoperability with Dual datatypes.

### Error in approximated rates and execution time

In [4]:
import numpy as np
MIN, MAX, SAMPLES, DAYS, d = 0, 4, 100000, 3, 1.0/365
c1 = np.random.rand(DAYS, SAMPLES) * (MAX - MIN) + MIN
c2 = np.random.rand(DAYS, SAMPLES) * (MAX - MIN) + MIN
r_true=((1 + d * (c1 + c2) / 100).prod(axis=0) - 1) * 100 / (d * DAYS)
c1_bar = ((1 + d * c1 / 100).prod(axis=0)**(1/DAYS) - 1) * 100 / d
c2_bar = ((1 + d * c2 / 100).prod(axis=0)**(1/DAYS) - 1) * 100 / d
r_bar = ((1 + d * (c1_bar + c2_bar) / 100) ** DAYS - 1) * 100 / (d * DAYS)
np.histogram(np.abs(r_true-r_bar), bins=[0, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1]) 

(array([ 3455,  3451, 22875, 21294, 48033,   892]),
 array([0.e+00, 5.e-07, 1.e-06, 5.e-06, 1.e-05, 5.e-05, 1.e+00]))

In [5]:
composite_curve = CompositeCurve(
    (
        Curve({dt(2022, 1, 1): 1.0, dt(2024, 1, 1): 0.95}, id="C1_"),
        Curve({dt(2022, 1, 1): 1.0, dt(2024, 1, 1): 0.99}, id="C2_"),
    )
)
%timeit composite_curve.rate(dt(2022, 6, 1), "1y", approximate=True)  

50.3 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
%timeit composite_curve.rate(dt(2022, 6, 1), "1y", approximate=False)

21.9 ms ± 890 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Curve operations: shift

In [7]:
curve = Curve({dt(2022, 1, 1): 1.0, dt(2023, 1, 1): 0.98})
print(curve.rate(dt(2022, 2, 1), "1d"))
print(curve.shift(50).rate(dt(2022, 2, 1), "1d"))

1.9926509362075961
2.4926509362108717


In [8]:
line_curve = LineCurve({dt(2022, 1, 1): 2.0, dt(2023, 1, 1): 2.6})
print(line_curve.rate(dt(2022, 2, 1), "1d"))
print(line_curve.shift(50).rate(dt(2022, 2, 1), "1d"))

2.050958904109589
2.550958904109589


### Curve operations: translate

In [9]:
for interpolation in [
    "linear", "log_linear", "linear_index", "flat_forward", "flat_backward", "linear_zero_rate"
]:
    curve = Curve(
        nodes={dt(2022, 1, 1): 1.0, dt(2022, 2, 1):0.998, dt(2022, 3, 1): 0.995}, 
        interpolation=interpolation
    )
    curve_translated = curve.translate(dt(2022, 1, 15)) 
    print(
        curve.rate(dt(2022, 2, 15), "1d"),
        curve_translated.rate(dt(2022, 2, 15), "1d") 
    )

3.8711064912719806 3.8711064912719806
3.8709012910311813 3.8709012910311813
3.8706902731000525 3.870690273092059
0.0 0.0
0.0 0.0
3.8971038951416404 3.9052558203165333


### Curve operations: roll

In [10]:
curve = Curve(
    nodes={dt(2022, 1, 1): 1.0, dt(2023, 1, 1): 0.98, dt(2024, 1, 1): 0.97},
    t=[dt(2022, 1, 1), dt(2022, 1, 1), dt(2022, 1, 1), dt(2022, 1, 1),
       dt(2023, 1, 1),
       dt(2024, 1, 1), dt(2024, 1, 1), dt(2024, 1, 1), dt(2024, 1, 1)]
)
print(curve.rate(dt(2022, 6, 1), "1d"))
print(curve.roll("30d").rate(dt(2022, 7, 1), "1d"))

2.1111503451809455
2.1111503451809455


In [11]:
line_curve = LineCurve(
    nodes={dt(2022, 1, 1): 2.0, dt(2023, 1, 1): 2.6, dt(2024, 1, 1): 2.5},
    t=[dt(2022, 1, 1), dt(2022, 1, 1), dt(2022, 1, 1), dt(2022, 1, 1),
       dt(2023, 1, 1),
       dt(2024, 1, 1), dt(2024, 1, 1), dt(2024, 1, 1), dt(2024, 1, 1)]
)
print(line_curve.rate(dt(2022, 6, 1)))
print(line_curve.roll("-31d").rate(dt(2022, 5, 1), "1d"))

2.3082258965546494
2.3082258965546494


### Operations on CompositeCurves

In [14]:
composite_curve.rate(dt(2022, 6, 1), "1d")

3.0252576094156325

In [15]:
composite_curve.shift(50).rate(dt(2022, 6, 1), "1d")

3.525257609418908

In [16]:
composite_curve.roll("30d").rate(dt(2022, 7, 1), "1d")

3.025257609407639

In [17]:
composite_curve.translate(dt(2022, 5, 1)).rate(dt(2022, 6, 1), "1d")

3.0252576094156325